In [ ]:
import math
import types
import uuid
from dotenv import load_dotenv
import os

from langchain.embeddings import init_embeddings
from langgraph.store.memory import InMemoryStore

from langgraph_bigtool import create_agent
from langgraph_bigtool.utils import (
    convert_positional_only_function_to_tool
)
from langchain_openai import ChatOpenAI

# Load environment variables (ie. API keys) from .env file
load_dotenv()

# Collect functions from `math` built-in
all_tools = []
for function_name in dir(math):
    function = getattr(math, function_name)
    if not isinstance(
        function, types.BuiltinFunctionType
    ):
        continue
    # This is an idiosyncrasy of the `math` library
    if tool := convert_positional_only_function_to_tool(
        function
    ):
        all_tools.append(tool)

# Create registry of tools. This is a dict mapping
# identifiers to tool instances.
tool_registry = {
    str(uuid.uuid4()): tool
    for tool in all_tools
}

# Index tool names and descriptions in the LangGraph Store
# Here we use a simple in-memory store, using OpenAI's text-embedding-3-small embedding model (make sure you have your OPENAI_API_KEY saved in your .env file)
embeddings = init_embeddings("openai:text-embedding-3-small")

store = InMemoryStore(
    index={
        "embed": embeddings,
        "dims": 1536,
        "fields": ["description"],
    }
)
for tool_id, tool in tool_registry.items():
    store.put(
        ("tools",),
        tool_id,
        {
            "description": f"{tool.name}: {tool.description}",
        },
    )

# Initialize agent
llm = ChatOpenAI(
    model="deepseek/deepseek-r1",
    api_key=os.environ["OPENROUTER_API_KEY"],
    base_url="https://openrouter.ai/api/v1"
)

builder = create_agent(llm, tool_registry)
agent = builder.compile(store=store)
print(agent)

C:\Users\leo_c\AppData\Local\Temp\ipykernel_55696\1597647674.py:29: LangChainBetaWarning: The function `convert_positional_only_function_to_tool` is in beta. It is actively being worked on, so the API may change.
  if tool := convert_positional_only_function_to_tool(
C:\Users\leo_c\AppData\Local\Temp\ipykernel_55696\1597647674.py:43: LangChainBetaWarning: The function `init_embeddings` is in beta. It is actively being worked on, so the API may change.
  embeddings = init_embeddings("openai:text-embedding-3-small")


In [2]:
query = "Use available tools to calculate arc cosine of 0.5."

# Test it out
for step in agent.stream(
    {"messages": query},
    stream_mode="updates",
):
    for _, update in step.items():
        for message in update.get("messages", []):
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  retrieve_tools (call_1krw21tnqubf094g5v2lnmqd)
 Call ID: call_1krw21tnqubf094g5v2lnmqd
  Args:
    query: arc cosine calculator
================================= Tool Message =================================

Available tools: ['cos', 'acos']
================================== Ai Message ==================================
Tool Calls:
  acos (call_qti8comwhr2gjk21rj85xxrc)
 Call ID: call_qti8comwhr2gjk21rj85xxrc
  Args:
    x: 0.5
================================= Tool Message =================================
Name: acos

1.0471975511965979
================================== Ai Message ==================================

The arc cosine of 0.5 is **1.0472 radians** (which equals **60 degrees**). 

This is because \(\cos(60^\circ) = 0.5\), so \(\arccos(0.5) = 60^\circ\) or \(\frac{\pi}{3}\) radians. 😊
